In [24]:
import scipy.io as scio 
dataFile = 'Cosine_Similarity.mat'
data = scio.loadmat(dataFile)

In [25]:
type(data)

dict

In [26]:
import numpy as np
res = np.array([np.array(item) for item in data.values()])

E:\Temp\ipykernel_24344\1659430892.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array([np.array(item) for item in data.values()])


In [27]:
res=res[3]

In [7]:
np.shape(res)

(12, 12, 1215)

In [28]:
print(res[:,:,0])

[[ 0.00000000e+00 -5.75432729e-02 -4.35899475e-01  4.78056453e-02
  -3.16962150e-01  2.84667033e-01 -5.02587647e-01 -4.01235797e-01
  -1.68646793e-01  3.74976071e-01  4.38590598e-01  1.80957224e-01]
 [-5.75432729e-02  0.00000000e+00  3.35463857e-01 -1.68803029e-01
   3.94167728e-01 -3.72456723e-01  4.99776276e-01  4.27219730e-01
   2.30126697e-01 -4.32570937e-01  6.42637486e-02 -4.34802518e-02]
 [-4.35899475e-01  3.35463857e-01  0.00000000e+00 -3.71031400e-01
   8.10602369e-01 -1.87755458e-01  7.21621983e-01  8.62800860e-01
  -1.85460818e-04 -6.60647326e-01 -2.98667686e-01  4.26940436e-02]
 [ 4.78056453e-02 -1.68803029e-01 -3.71031400e-01  0.00000000e+00
   1.20467832e-01 -1.98995344e-01 -1.10471071e-01  2.32754438e-02
   6.93706420e-02  2.80261818e-01  3.83667721e-01 -5.72101330e-01]
 [-3.16962150e-01  3.94167728e-01  8.10602369e-01  1.20467832e-01
   0.00000000e+00 -2.12226018e-01  7.46688543e-01  8.97401698e-01
  -5.05639965e-02 -7.17196251e-01 -2.50594249e-01  1.88708994e-02]
 [ 2.

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim


In [44]:
data=[]
for i in range(0,np.shape(res)[2]):
    data.append(res[:,:,i]) 
data=np.array(data)
data.shape

(1215, 12, 12)

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim

# Step 1: Preprocess the dataset if necessary

# Step 2: Reshape the dataset
reshaped_dataset = data

# Step 3: Split the dataset into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * len(reshaped_dataset))
train_data = reshaped_dataset[:train_size]
test_data = reshaped_dataset[train_size:]

# Step 4: Define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(12*12, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 12*12),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Step 5: Train the autoencoder
autoencoder = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

num_epochs = 100
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch in train_loader:
        inputs = batch.view(batch.size(0), -1).float()
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()



In [78]:
inputs = batch.view(batch.size(0), -1).float()

In [57]:
batch.size(0)

12

In [79]:
data.size(0)
data.view(data.size(0), -1)

tensor([[ 0.0000, -0.0575, -0.4359,  ...,  0.2327,  0.3534,  0.0000],
        [ 0.0000, -0.0575, -0.4359,  ...,  0.2327,  0.3534,  0.0000],
        [ 0.0000, -0.0575, -0.4359,  ...,  0.2327,  0.3534,  0.0000],
        ...,
        [ 0.0000, -0.5378, -0.4907,  ..., -0.3461, -0.3002,  0.0000],
        [ 0.0000, -0.5398, -0.4916,  ..., -0.3830, -0.3159,  0.0000],
        [ 0.0000, -0.5398, -0.4916,  ..., -0.3830, -0.3159,  0.0000]],
       dtype=torch.float64)

In [80]:
# Step 6: Embed the whole dataset
all_inputs=data.view(data.shape[0],-1).float()
# Save the embedded representations to a file
embedded_representations = autoencoder.encoder(all_inputs)
#torch.save(embedded_representations, 'embedded_representations.pth')


In [81]:


result = embedded_representations.detach().numpy()
 

In [82]:
import pandas as pd
result=pd.DataFrame(result)

In [84]:
result.to_csv("graph_features_new.csv",index=False)

In [16]:
res=[]
for i in range(0,np.shape(res)[2]):
    res.append(res[:,:,i]) 
res.shape

[array([ 0.        , -0.05754327, -0.43589948,  0.04780565, -0.31696215,
         0.28466703, -0.50258765, -0.4012358 , -0.16864679,  0.37497607,
         0.4385906 ,  0.18095722]),
 array([ 0.        , -0.05754327, -0.43589948,  0.04780565, -0.31696215,
         0.28466703, -0.50258765, -0.4012358 , -0.16864679,  0.37497607,
         0.4385906 ,  0.18095722]),
 array([ 0.        , -0.05754327, -0.43589948,  0.04780565, -0.31696215,
         0.28466703, -0.50258765, -0.4012358 , -0.16864679,  0.37497607,
         0.4385906 ,  0.18095722]),
 array([ 0.        , -0.05754327, -0.43589948,  0.04780565, -0.31696215,
         0.28466703, -0.50258765, -0.4012358 , -0.16864679,  0.37497607,
         0.4385906 ,  0.18095722]),
 array([ 0.        , -0.05754327, -0.43589948,  0.04780565, -0.31696215,
         0.28466703, -0.50258765, -0.4012358 , -0.16864679,  0.37497607,
         0.4385906 ,  0.18095722]),
 array([ 0.        , -0.05754327, -0.43589948,  0.04780565, -0.31696215,
         0.284667

In [17]:
import pandas as pd
china=pd.DataFrame(china)

In [20]:
china.to_csv('graph_features.csv',index=False)